# Parameter Preparation - Interactive Notebook

This notebook provides **4 pre-configured parameter templates** for different use cases.

## Quick Guide

1. **Choose your template** (Template 1-4 below)
2. **Customize the parameters** directly in the dictionary
3. **Run the cell** to see your configuration
4. **Execute the preparation** using the last cell

---

In [1]:
import os
import sys
from pathlib import Path

project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from prepare_params import prepare_production_params

print("Modules imported successfully")
print(f"Current working directory: {os.getcwd()}")


kml = "Sample Points\AfforestationSItesFixed.kml"
shp = "Sample Points\FieldPoints32_2018.shp"

print(kml)
print(shp)

Modules imported successfully
Current working directory: C:\Users\aradar\LEAF Files\LEAF_runner
Sample Points\AfforestationSItesFixed.kml
Sample Points\FieldPoints32_2018.shp


In [2]:
# TEMPLATE 1: Single Year Monthly Mosaics
# Use Case: Generate monthly mosaics for specific months in one year
# Example: Summer months (June, July, August) of 2023
# Customize these parameters as needed:

Template1_ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',                    # Sensor type
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    'year': 2023,                          # CHANGE THIS: Year to process
    'months': [6, 7, 8],                   # CHANGE THIS: Months to process (1-12)
    
    # ============ REGION PARAMETERS ============
    'regions': kml,  # CHANGE THIS: Path to your KML/SHP file
    'regions_start_index': 21,              # CHANGE THIS: Start at this region index
    'regions_end_index': 30,             # CHANGE THIS: End at this index (None = all)
    
    # ============ BUFFER PARAMETERS ============
    'spatial_buffer_m': -20,             # UNCOMMENT & CHANGE: Buffer in meters around regions
    # 'temporal_buffer': [-5, 5],          # UNCOMMENT & CHANGE: [days_before, days_after]
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 30,                      # Resolution in meters
    'projection': 'EPSG:3979',             # Coordinate projection
    'prod_names': ['mosaic'],    #['LAI', 'fCOVER', 'fAPAR', 'Albedo'], 
    'out_folder': 'E:/S2_mosaics_runner_2026/S2_temp1_3_30m',
    'out_datatype': 'int16'
}

Template1_CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}

# Display configuration
print("="*80)
print("TEMPLATE 1: Single Year Monthly Mosaics")
print("="*80)
print(f"Year: {Template1_ProdParams['year']}")
print(f"Months: {Template1_ProdParams['months']}")
print(f"Region file: {Template1_ProdParams['regions']}")
print(f"Indices: {Template1_ProdParams['regions_start_index']} to {Template1_ProdParams['regions_end_index'] or 'end'}")
print(f"Output: {Template1_ProdParams['out_folder']}")
if 'spatial_buffer_m' in Template1_ProdParams:
    print(f"Spatial buffer: {Template1_ProdParams['spatial_buffer_m']}m")
if 'temporal_buffer' in Template1_ProdParams:
    print(f"Temporal buffer: {Template1_ProdParams['temporal_buffer']} days")
print("="*80)
print("\n✓ Template 1 configured")

TEMPLATE 1: Single Year Monthly Mosaics
Year: 2023
Months: [6, 7, 8]
Region file: Sample Points\AfforestationSItesFixed.kml
Indices: 21 to 30
Output: E:/S2_mosaics_runner_2026/S2_temp1_3_30m
Spatial buffer: -20m

✓ Template 1 configured


In [3]:
# TEMPLATE 2: Custom Date Ranges
#Use Case: Specify exact start and end dates for each time window
#Example: Pre/post event comparison or specific growing periods
#What this creates:
#Custom windows based on your exact dates
# Customize these parameters as needed:

Template2_ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    
    # CHANGE THESE: Specify exact date ranges (must have same number of start and end dates)
    'start_dates': ['2023-06-01', '2023-08-15', '2023-10-01'],  # Start dates for each window
    'end_dates':   ['2023-06-30', '2023-09-15', '2023-10-31'],  # End dates for each window
    
    # ============ REGION PARAMETERS ============
    'regions': kml,  # 🔧 CHANGE THIS
    'regions_start_index': 1,              # CHANGE THIS
    'regions_end_index': 5,               # CHANGE THIS: Process first 10 regions only
    
    # ============ BUFFER PARAMETERS ============
    'spatial_buffer_m': 100,               # CHANGE THIS: 100m buffer
    # 'temporal_buffer': [-3, 3],          # UNCOMMENT & CHANGE if needed
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 30,
    'projection': 'EPSG:3979',
    'prod_names': ['mosaic'],    #['LAI', 'fCOVER', 'fAPAR', 'Albedo'], 
    'out_folder': 'E:/S2_mosaics_runner_2026/S2_temp2_30m',
    'out_datatype': 'int16'
}

Template2_CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}

# Display configuration
print("="*80)
print("TEMPLATE 2: Custom Date Ranges")
print("="*80)
print(f"Number of windows: {len(Template2_ProdParams['start_dates'])}")
for i, (start, end) in enumerate(zip(Template2_ProdParams['start_dates'], Template2_ProdParams['end_dates']), 1):
    print(f"  Window {i}: {start} to {end}")
print(f"Region file: {Template2_ProdParams['regions']}")
print(f"Indices: {Template2_ProdParams['regions_start_index']} to {Template2_ProdParams['regions_end_index'] or 'end'}")
print(f"Spatial buffer: {Template2_ProdParams.get('spatial_buffer_m', 'None')}m")
print(f"Output: {Template2_ProdParams['out_folder']}")
print("="*80)
print("\n Template 2 configured")

TEMPLATE 2: Custom Date Ranges
Number of windows: 3
  Window 1: 2023-06-01 to 2023-06-30
  Window 2: 2023-08-15 to 2023-09-15
  Window 3: 2023-10-01 to 2023-10-31
Region file: Sample Points\AfforestationSItesFixed.kml
Indices: 1 to 5
Spatial buffer: 100m
Output: E:/S2_mosaics_runner_2026/S2_temp2_30m

 Template 2 configured


In [4]:
# TEMPLATE 3: Single Date Processing
# Use Case: Process imagery for one specific date
# Example: Capture a snapshot on a specific event date
# Customize these parameters as needed:

Template3_ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    'start_dates': ['2023-07-15'],            # CHANGE THIS: Single date to process (YYYY-MM-DD)
    
    # ============ REGION PARAMETERS ============
    'regions': kml,  # 🔧 CHANGE THIS
    'regions_start_index': 0,
    'regions_end_index': 5,                # CHANGE THIS: First 5 regions only
    
    # ============ BUFFER PARAMETERS ============
    'spatial_buffer_m': 50,                # CHANGE THIS: 50m buffer
    'temporal_buffer': [-20, 10],            # CHANGE THIS: bufferdays around the date
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 30,
    'projection': 'EPSG:3979',
    'prod_names': ['mosaic'],    #['LAI', 'fCOVER', 'fAPAR', 'Albedo'], 
    'out_folder': 'E:/S2_mosaics_runner_2026/S2_temp3_30m',
    'out_datatype': 'int16'
}

Template3_CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}

# Display configuration
print("="*80)
print("TEMPLATE 3: Single Date Processing")
print("="*80)
print(f"Date: {Template3_ProdParams['start_dates']}")
print(f"Region file: {Template3_ProdParams['regions']}")
print(f"Indices: {Template3_ProdParams['regions_start_index']} to {Template3_ProdParams['regions_end_index'] or 'end'}")
print(f"Spatial buffer: {Template3_ProdParams.get('spatial_buffer_m', 'None')}m")
print(f"Temporal buffer: {Template3_ProdParams.get('temporal_buffer', 'None')} days")
print(f"Output: {Template3_ProdParams['out_folder']}")
print("="*80)
print("\n Template 3 configured")

TEMPLATE 3: Single Date Processing
Date: ['2023-07-15']
Region file: Sample Points\AfforestationSItesFixed.kml
Indices: 0 to 5
Spatial buffer: 50m
Temporal buffer: [-20, 10] days
Output: E:/S2_mosaics_runner_2026/S2_temp3_30m

 Template 3 configured


In [5]:
# TEMPLATE 4: Multi-Year Monthly Mosaics
# Use Case: Repeat the same months across multiple years for trend analysis
# Example: Summer months for 5 years (2019-2023)
# Customize these parameters as needed:

Template4_ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',
    'year': 2019,                          # CHANGE THIS: Starting year
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    'num_years': 5,                        # CHANGE THIS: Number of years to process
    'months': [7, 8],                   # CHANGE THIS: Months to repeat each year
    
    # ============ REGION PARAMETERS ============
    'regions': kml,  # 🔧 CHANGE THIS
    'regions_start_index': 1,
    'regions_end_index': 3,             # Process all regions
    
    # ============ BUFFER PARAMETERS ============
    'spatial_buffer_m': 100,               # CHANGE THIS
    'temporal_buffer': [-5, 5],            # CHANGE THIS: ±5 days
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 30,
    'projection': 'EPSG:3979',
    'prod_names': ['mosaic'],    #['LAI', 'fCOVER', 'fAPAR', 'Albedo'], 
    'out_folder': 'E:/S2_mosaics_runner_2026/S2_temp4_30m',
    'out_datatype': 'int16'
}

Template4_CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}

# Display configuration
print("="*80)
print("TEMPLATE 4: Multi-Year Monthly Mosaics")
print("="*80)
print(f"Years: {Template4_ProdParams['year']} to {Template4_ProdParams['year'] + Template4_ProdParams['num_years'] - 1}")
print(f"Months per year: {Template4_ProdParams['months']}")
print(f"Total windows: {len(Template4_ProdParams['months']) * Template4_ProdParams['num_years']}")
print(f"Region file: {Template4_ProdParams['regions']}")
print(f"Indices: {Template4_ProdParams['regions_start_index']} to {Template4_ProdParams['regions_end_index'] or 'end'}")
print(f"Spatial buffer: {Template4_ProdParams.get('spatial_buffer_m', 'None')}m")
print(f"Temporal buffer: {Template4_ProdParams.get('temporal_buffer', 'None')} days")
print(f"Output: {Template4_ProdParams['out_folder']}")
print("="*80)
print("\n Template 4 configured")

TEMPLATE 4: Multi-Year Monthly Mosaics
Years: 2019 to 2023
Months per year: [7, 8]
Total windows: 10
Region file: Sample Points\AfforestationSItesFixed.kml
Indices: 1 to 3
Spatial buffer: 100m
Temporal buffer: [-5, 5] days
Output: E:/S2_mosaics_runner_2026/S2_temp4_30m

 Template 4 configured


---

# Run Parameter Preparation

Choose which template to run by uncommenting the appropriate line below:

In [6]:
# ============================================
# SELECT YOUR TEMPLATE HERE
# ============================================
# Uncomment ONE of these lines:

ProdParams, CompParams = Template1_ProdParams, Template1_CompParams  # Monthly mosaics
# ProdParams, CompParams = Template2_ProdParams, Template2_CompParams  # Custom date ranges
# ProdParams, CompParams = Template3_ProdParams, Template3_CompParams  # Single date
# ProdParams, CompParams = Template4_ProdParams, Template4_CompParams  # Multi-year monthly

print("Template selected")
print(f"  Output folder: {ProdParams['out_folder']}")

Template selected
  Output folder: E:/S2_mosaics_runner_2026/S2_temp1_3_30m


## Execute Parameter Preparation

This cell will:
1. Load regions from your KML/SHP file
2. Generate time windows
3. Validate all polygons
4. Create a processing log
5. Return validated parameters ready for Production.py

In [7]:
# Run the parameter preparation
result = prepare_production_params(ProdParams, CompParams)

if result:
    print("\n" + "="*80)
    print(" SUCCESS - Parameters validated and ready!")
    print("="*80)
    print(f"\n Summary:")
    print(f"  Regions: {len(result['ProdParams']['regions'])}")
    print(f"  Time windows: {len(result['ProdParams']['start_dates'])}")
    print(f"  Total tasks: {len(result['ProdParams']['regions']) * len(result['ProdParams']['start_dates'])}")
    print(f"  Output folder: {result['ProdParams']['out_folder']}")
    print(f"\n Processing log saved to: {result['ProdParams']['out_folder']}/polygon_processing_log.csv")
    print("\n" + "="*80)
    print("Next step: Call Production.py with the validated parameters")
    print("="*80)
else:
    print("\n" + "="*80)
    print(" FAILED - Parameter validation failed")
    print("="*80)
    print("Check the error messages above for details.")


PARAMETER PREPARATION PIPELINE

Step 1/3: Loading regions from files...
<handle_regions_from_file> Detected file-based regions input: Sample Points\AfforestationSItesFixed.kml
<handle_regions_from_file> Loading regions...
Applied -20m buffer to geometries
<handle_regions_from_file> Loaded 10 regions from file

Step 2/3: Generating temporal windows...
<form_time_windows> Generating temporal windows...
<form_time_windows> Mode 1: Single year months (2023)
<form_time_windows> Generated 3 time windows

Step 3/3: Validating polygons...

POLYGON VALIDATION AND FILTERING

Processing log saved to: E:/S2_mosaics_runner_2026/S2_temp1_3_30m\polygon_processing_log.csv

Summary:
   Total region-date combinations: 30
   Valid for processing: 30
   Skipped (zero area): 0
{'region21': {'type': 'Polygon', 'coordinates': [[[-75.9688683503077, 44.74555787917692], [-75.96867783588804, 44.745730532884465], [-75.96867608081142, 44.7457321424386], [-75.96840328883466, 44.74598531513747], [-75.96809896800484

---

# View Results (Optional)

Examine the validated parameters and processing log:

In [8]:
# View validated parameters
if result:
    print("\n Time Windows:")
    print("=" * 80)
    for i, (start, end) in enumerate(zip(result['ProdParams']['start_dates'], 
                                          result['ProdParams']['end_dates']), 1):
        print(f"  {i:2d}. {start} to {end}")
    
    print("\n Regions:")
    print("=" * 80)
    region_names = list(result['ProdParams']['regions'].keys())
    for i, name in enumerate(region_names[:10], 1):  # Show first 10
        print(f"  {i:2d}. {name}")
    if len(region_names) > 10:
        print(f"  ... and {len(region_names) - 10} more")


 Time Windows:
   1. 2023-06-01 to 2023-06-30
   2. 2023-07-01 to 2023-07-31
   3. 2023-08-01 to 2023-08-31

 Regions:
   1. region21
   2. region22
   3. region23
   4. region24
   5. region25
   6. region26
   7. region27
   8. region28
   9. region29
  10. region30


In [9]:
# View processing log (if pandas is available)
try:
    import pandas as pd
    if result and result['processing_log'] is not None:
        print("\n Processing Log Preview:")
        print("=" * 80)
        display(result['processing_log'].head(10))
        print(f"\nTotal entries: {len(result['processing_log'])}")
except ImportError:
    print("pandas not available - install it to view the processing log in this cell")
    print("You can still view the CSV file directly in the output folder")


 Processing Log Preview:


,region_id,date,area_m2,will_process,status,skip_reason,timestamp
0,region21,2023-06-30,1086.87,True,QUEUED,N/A,2026-02-09T11:58:02.836770
1,region21,2023-07-31,1086.87,True,QUEUED,N/A,2026-02-09T11:58:02.836770
2,region21,2023-08-31,1086.87,True,QUEUED,N/A,2026-02-09T11:58:02.836770
3,region22,2023-06-30,53090.95,True,QUEUED,N/A,2026-02-09T11:58:02.836770
4,region22,2023-07-31,53090.95,True,QUEUED,N/A,2026-02-09T11:58:02.836770
5,region22,2023-08-31,53090.95,True,QUEUED,N/A,2026-02-09T11:58:02.836770
6,region23,2023-06-30,38987.64,True,QUEUED,N/A,2026-02-09T11:58:02.836770
7,region23,2023-07-31,38987.64,True,QUEUED,N/A,2026-02-09T11:58:02.836770
8,region23,2023-08-31,38987.64,True,QUEUED,N/A,2026-02-09T11:58:02.836770
9,region24,2023-06-30,14877.86,True,QUEUED,N/A,2026-02-09T11:58:02.836770



Total entries: 30


---

# Next Steps

Now that your parameters are validated, you can:

1. **Review the processing log** in your output folder
2. **Call Production.py** with the validated parameters:

```python
from Production import main
main(result['ProdParams'], result['CompParams'])
```

---

# Parameter Reference

### Common Parameters (All Templates)

| Parameter | Type | Description | Example |
|-----------|------|-------------|----------|
| `sensor` | str | Sensor type | `'HLS_SR'` |
| `regions` | str | Path to KML/SHP file | `'./data/regions.kml'` |
| `regions_start_index` | int | First region to process | `0` |
| `regions_end_index` | int/None | Last region to process | `10` or `None` |
| `spatial_buffer_m` | float | Buffer in meters | `100` |
| `temporal_buffer` | list | Days before/after | `[-5, 5]` |
| `resolution` | int | Meters per pixel | `30` |
| `projection` | str | Coordinate system | `'EPSG:3979'` |
| `out_folder` | str | Output directory | `'./output'` |

### Template-Specific Parameters

**Template 1 & 4:**
- `year`: Starting year (int)
- `months`: List of months (list of int, 1-12)
- `num_years`: Number of years (Template 4 only)

**Template 2:**
- `start_dates`: List of start dates (list of str, 'YYYY-MM-DD')
- `end_dates`: List of end dates (list of str, 'YYYY-MM-DD')

**Template 3:**
- `start_date`: Single date (str, 'YYYY-MM-DD')

---

## Tips

1. **Test with small subsets first**: Use `regions_end_index` to process just a few regions
2. **Use buffers wisely**: Temporal buffers increase image availability
3. **Check the log**: Always review `polygon_processing_log.csv` before full runs
4. **Save working configs**: Keep successful parameter sets for future use

---

In [10]:
import os
import sys
import geopandas as gpd

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
source_path = os.path.join(cwd, 'source')
sys.path.append(source_path)
sys.path

from Production import main


In [11]:
INVALID_KEYS = {
    "regions_start_index",
    "regions_end_index",
    "spatial_buffer_m",
    "temporal_buffer",
    "num_years"
}

for key in list(ProdParams.keys()):
    if key in INVALID_KEYS:
        ProdParams.pop(key)


In [12]:
print(ProdParams)

{'sensor': 'S2_SR', 'unit': 2, 'nbYears': -1, 'year': 2023, 'months': [6, 7, 8], 'regions': {'region21': {'type': 'Polygon', 'coordinates': [[[-75.9688683503077, 44.74555787917692], [-75.96867783588804, 44.745730532884465], [-75.96867608081142, 44.7457321424386], [-75.96840328883466, 44.74598531513747], [-75.96809896800484, 44.746206787628694], [-75.96766446445768, 44.74647454548249], [-75.96764410323905, 44.74648862179537], [-75.96762613175679, 44.74650426357531], [-75.96755193662203, 44.7465769583314], [-75.96772955931438, 44.74635307263437], [-75.9680529332554, 44.74611199591419], [-75.96807186850452, 44.74609609865566], [-75.9687190595175, 44.74548275331839], [-75.9688683503077, 44.74555787917692]]]}, 'region22': {'type': 'Polygon', 'coordinates': [[[-75.64377355388552, 45.218410413010425], [-75.64377005656662, 45.218407629005014], [-75.64375745210864, 45.21815305284673], [-75.64371601564736, 45.2170598078128], [-75.64997605911182, 45.21442934021909], [-75.65017759115877, 45.214714

In [13]:
main(ProdParams, CompParams)

<MosaicProduction> User defined input parameters:
sensor: S2_SR
unit: 2
nbYears: -1
year: 2023
months: [6, 7, 8]
regions: {'region21': {'type': 'Polygon', 'coordinates': [[[-75.9688683503077, 44.74555787917692], [-75.96867783588804, 44.745730532884465], [-75.96867608081142, 44.7457321424386], [-75.96840328883466, 44.74598531513747], [-75.96809896800484, 44.746206787628694], [-75.96766446445768, 44.74647454548249], [-75.96764410323905, 44.74648862179537], [-75.96762613175679, 44.74650426357531], [-75.96755193662203, 44.7465769583314], [-75.96772955931438, 44.74635307263437], [-75.9680529332554, 44.74611199591419], [-75.96807186850452, 44.74609609865566], [-75.9687190595175, 44.74548275331839], [-75.9688683503077, 44.74555787917692]]]}, 'region22': {'type': 'Polygon', 'coordinates': [[[-75.64377355388552, 45.218410413010425], [-75.64377005656662, 45.218407629005014], [-75.64375745210864, 45.21815305284673], [-75.64371601564736, 45.2170598078128], [-75.64997605911182, 45.21442934021909], 